In [ ]:
""" pandas review point
1) 조인, 합성 관련 메서드
2) 통계값 관련 메서드
3) 시각화 관련 메서드
4) 전처리, 피처 엔지니어링 메서드 
"""

In [1]:
import warnings
import numpy as np
import pandas as pd

In [2]:
""" binary classification binary_classification
resource from:
    https://www.kaggle.com/datasets/nelgiriyewithana/credit-card-fraud-detection-dataset-2023
"""

PATH = "binary_classification/creditcard_2023/creditcard_2023.csv"
df = pd.read_csv(PATH)
df

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568625,568625,-0.833437,0.061886,-0.899794,0.904227,-1.002401,0.481454,-0.370393,0.189694,-0.938153,...,0.167503,0.419731,1.288249,-0.900861,0.560661,-0.006018,3.308968,0.081564,4394.16,1
568626,568626,-0.670459,-0.202896,-0.068129,-0.267328,-0.133660,0.237148,-0.016935,-0.147733,0.483894,...,0.031874,0.388161,-0.154257,-0.846452,-0.153443,1.961398,-1.528642,1.704306,4653.40,1
568627,568627,-0.311997,-0.004095,0.137526,-0.035893,-0.042291,0.121098,-0.070958,-0.019997,-0.122048,...,0.140788,0.536523,-0.211100,-0.448909,0.540073,-0.755836,-0.487540,-0.268741,23572.85,1
568628,568628,0.636871,-0.516970,-0.300889,-0.144480,0.131042,-0.294148,0.580568,-0.207723,0.893527,...,-0.060381,-0.195609,-0.175488,-0.554643,-0.099669,-1.434931,-0.159269,-0.076251,10160.83,1


In [3]:
""" check the basic statistic information about dataframe """

df.info(), df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568630 entries, 0 to 568629
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      568630 non-null  int64  
 1   V1      568630 non-null  float64
 2   V2      568630 non-null  float64
 3   V3      568630 non-null  float64
 4   V4      568630 non-null  float64
 5   V5      568630 non-null  float64
 6   V6      568630 non-null  float64
 7   V7      568630 non-null  float64
 8   V8      568630 non-null  float64
 9   V9      568630 non-null  float64
 10  V10     568630 non-null  float64
 11  V11     568630 non-null  float64
 12  V12     568630 non-null  float64
 13  V13     568630 non-null  float64
 14  V14     568630 non-null  float64
 15  V15     568630 non-null  float64
 16  V16     568630 non-null  float64
 17  V17     568630 non-null  float64
 18  V18     568630 non-null  float64
 19  V19     568630 non-null  float64
 20  V20     568630 non-null  float64
 21  V21     56

(None,
                   id            V1            V2            V3            V4  \
 count  568630.000000  5.686300e+05  5.686300e+05  5.686300e+05  5.686300e+05   
 mean   284314.500000 -5.638058e-17 -1.319545e-16 -3.518788e-17 -2.879008e-17   
 std    164149.486122  1.000001e+00  1.000001e+00  1.000001e+00  1.000001e+00   
 min         0.000000 -3.495584e+00 -4.996657e+01 -3.183760e+00 -4.951222e+00   
 25%    142157.250000 -5.652859e-01 -4.866777e-01 -6.492987e-01 -6.560203e-01   
 50%    284314.500000 -9.363846e-02 -1.358939e-01  3.528579e-04 -7.376152e-02   
 75%    426471.750000  8.326582e-01  3.435552e-01  6.285380e-01  7.070047e-01   
 max    568629.000000  2.229046e+00  4.361865e+00  1.412583e+01  3.201536e+00   
 
                  V5            V6            V7            V8            V9  \
 count  5.686300e+05  5.686300e+05  5.686300e+05  5.686300e+05  5.686300e+05   
 mean   7.997245e-18 -3.958636e-17 -3.198898e-17  2.109273e-17  3.998623e-17   
 std    1.000001e+00  

In [5]:
""" multi-index by using set_index() """

df.set_index(['V1', 'V2'], inplace=True)  # set the multi-index
df.reset_index(inplace=True, drop=True)  # undo the multi-index

In [15]:
""" use fillna, fillna drop """

df.dropna(subset=df.columns, inplace=True)
df

,V1,V2,id,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.260648,-0.469648,0,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,0.985100,-0.356045,1,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,-0.260272,-0.949385,2,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,-0.152152,-0.508959,3,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,-0.206820,-0.165280,4,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568625,-0.833437,0.061886,568625,-0.899794,0.904227,-1.002401,0.481454,-0.370393,0.189694,-0.938153,...,0.167503,0.419731,1.288249,-0.900861,0.560661,-0.006018,3.308968,0.081564,4394.16,1
568626,-0.670459,-0.202896,568626,-0.068129,-0.267328,-0.133660,0.237148,-0.016935,-0.147733,0.483894,...,0.031874,0.388161,-0.154257,-0.846452,-0.153443,1.961398,-1.528642,1.704306,4653.40,1
568627,-0.311997,-0.004095,568627,0.137526,-0.035893,-0.042291,0.121098,-0.070958,-0.019997,-0.122048,...,0.140788,0.536523,-0.211100,-0.448909,0.540073,-0.755836,-0.487540,-0.268741,23572.85,1
568628,0.636871,-0.516970,568628,-0.300889,-0.144480,0.131042,-0.294148,0.580568,-0.207723,0.893527,...,-0.060381,-0.195609,-0.175488,-0.554643,-0.099669,-1.434931,-0.159269,-0.076251,10160.83,1


In [29]:
""" inner join, outer join, left join, right join """


df1 = pd.DataFrame({
    'ID': [1, 2, 3, 4],
    'Name': ['Alice', 'Bob', 'Charlie', 'David']
})
df2 = pd.DataFrame({
    'ID': [3, 4, 5, 6],
    'Age': [24, 27, 22, 32]
})

# natural join: 교집합, 동시에 존재하는 키값만 결과에 포함
inner_result = pd.merge(
    df1, 
    df2,
    how="inner",
    on="ID"  # natural join
)

# outer join: 합집합
outer_result = pd.merge(
    df1,
    df2,
    how="outer",
    on="ID"
)

# left join: 왼쪽 데이터프레임 기준으로 병합
left_result = pd.merge(
    df1,
    df2,
    how="left",
    on="ID"
)

# right join: 오른쪽 데이터프레임 기준 병합
right_result = pd.merge(
    df1,
    df2,
    how="right",
    on="ID"
)

# concat: 단순 병합
# 병합 축 설정에 따라서 결과 상이
concat_result = pd.concat(
    [df1, df2],
    axis=0
)

In [20]:
inner_result

,ID,Name,Age
0,3,Charlie,24
1,4,David,27


In [22]:
outer_result

,ID,Name,Age
0,1,Alice,NaN
1,2,Bob,NaN
2,3,Charlie,24.0
3,4,David,27.0
4,5,NaN,22.0
5,6,NaN,32.0


In [24]:
left_result

,ID,Name,Age
0,1,Alice,NaN
1,2,Bob,NaN
2,3,Charlie,24.0
3,4,David,27.0


In [26]:
right_result

,ID,Name,Age
0,3,Charlie,24
1,4,David,27
2,5,NaN,22
3,6,NaN,32


In [30]:
concat_result

,ID,Name,Age
0,1,Alice,NaN
1,2,Bob,NaN
2,3,Charlie,NaN
3,4,David,NaN
0,3,NaN,24.0
1,4,NaN,27.0
2,5,NaN,22.0
3,6,NaN,32.0


In [31]:
""" data type casting """

df['V1'] = df['V1'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568630 entries, 0 to 568629
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      568630 non-null  int64  
 1   V2      568630 non-null  float64
 2   id      568630 non-null  int64  
 3   V3      568630 non-null  float64
 4   V4      568630 non-null  float64
 5   V5      568630 non-null  float64
 6   V6      568630 non-null  float64
 7   V7      568630 non-null  float64
 8   V8      568630 non-null  float64
 9   V9      568630 non-null  float64
 10  V10     568630 non-null  float64
 11  V11     568630 non-null  float64
 12  V12     568630 non-null  float64
 13  V13     568630 non-null  float64
 14  V14     568630 non-null  float64
 15  V15     568630 non-null  float64
 16  V16     568630 non-null  float64
 17  V17     568630 non-null  float64
 18  V18     568630 non-null  float64
 19  V19     568630 non-null  float64
 20  V20     568630 non-null  float64
 21  V21     56

In [33]:
""" use groupby """

df3 = pd.DataFrame({
    'Category': ['A', 'A', 'B', 'B', 'C', 'C', 'A', 'B'],
    'Type': ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'],
    'Value1': [10, 20, 10, 40, 50, 60, 70, 80],
    'Value2': [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5]
})
df3

,Category,Type,Value1,Value2
0,A,one,10,1.5
1,A,two,20,2.5
2,B,one,10,3.5
3,B,two,40,4.5
4,C,one,50,5.5
5,C,two,60,6.5
6,A,one,70,7.5
7,B,two,80,8.5


In [37]:
df3.groupby(['Category', "Type"]).agg({
    "Value1": ["sum", "mean", "std"],
    "Value2": ["sum", "mean", "std"]
})

Value1                  Value2               
                 sum  mean        std    sum mean       std
Category Type                                              
A        one      80  40.0  42.426407    9.0  4.5  4.242641
         two      20  20.0        NaN    2.5  2.5       NaN
B        one      10  10.0        NaN    3.5  3.5       NaN
         two     120  60.0  28.284271   13.0  6.5  2.828427
C        one      50  50.0        NaN    5.5  5.5       NaN
         two      60  60.0        NaN    6.5  6.5       NaN

In [36]:
df3.groupby(['Category']).agg({
    "Value1": ["sum", "mean", "std"],
    "Value2": ["sum", "mean", "std"]
})

Value1                       Value2                    
            sum       mean        std    sum      mean       std
Category                                                        
A           100  33.333333  32.145503   11.5  3.833333  3.214550
B           130  43.333333  35.118846   16.5  5.500000  2.645751
C           110  55.000000   7.071068   12.0  6.000000  0.707107

In [39]:
df3.groupby(["Category"])[["Value1", "Value2"]].sum()

,Value1,Value2
Category,,
A,100,11.5
B,130,16.5
C,110,12.0


In [63]:
sum_df = df3.groupby(["Category"])[["Value1", "Value2"]]
sum_df.size()

Category
A    3
B    3
C    2
dtype: int64

In [67]:
filter_df = df3.groupby(["Category"]).filter(lambda x: x.Value1.sum() >= 130)
filter_df

,Category,Type,Value1,Value2
2,B,one,10,3.5
3,B,two,40,4.5
7,B,two,80,8.5


In [66]:
filter_df.groupby(["Category"])[["Value1", "Value2"]].sum()

,Value1,Value2
Category,,
A,100,11.5
B,130,16.5
C,110,12.0


In [68]:
df3.groupby(["Category", "Type"]).mean()

Value1  Value2
Category Type                
A        one     40.0     4.5
         two     20.0     2.5
B        one     10.0     3.5
         two     60.0     6.5
C        one     50.0     5.5
         two     60.0     6.5

In [70]:
df3.groupby(["Category", "Type"])[["Value1"]].sum().unstack()

Value1     
Type        one  two
Category            
A            80   20
B            10  120
C            50   60